In [1]:
print('What\'s up?') # In Python, \ scapes the following character.
                     # That's the reason it's not used in file paths

What's up?


# Notebook Imports

In [2]:
from os import walk
from os.path import join

import pandas as pd

# Constants

In [3]:
EXAMPLE_FILE = 'SpamData/01_Processing/practice_email.txt'

SPAM_1_PATH = 'SpamData/01_Processing/spam_assassin_corpus/spam_1'
SPAM_2_PATH = 'SpamData/01_Processing/spam_assassin_corpus/spam_2'
EASY_NONSPAM_1_PATH = 'SpamData/01_Processing/spam_assassin_corpus/easy_ham_1'
EASY_NONSPAM_2_PATH = 'SpamData/01_Processing/spam_assassin_corpus/easy_ham_2'

SPAM_CAT = 1
HAM_CAT = 0

# Reading Files

In [4]:
stream = open(EXAMPLE_FILE, encoding='latin-1')
message = stream.read()
stream.close()

print(type(stream))
print(type(message))
print(message)

<class '_io.TextIOWrapper'>
<class 'str'>
From exmh-workers-admin@redhat.com  Thu Aug 22 12:36:23 2002
Return-Path: <exmh-workers-admin@spamassassin.taint.org>
Delivered-To: zzzz@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id D03E543C36
	for <zzzz@localhost>; Thu, 22 Aug 2002 07:36:16 -0400 (EDT)
Received: from phobos [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 12:36:16 +0100 (IST)
Received: from listman.spamassassin.taint.org (listman.spamassassin.taint.org [66.187.233.211]) by
    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g7MBYrZ04811 for
    <zzzz-exmh@spamassassin.taint.org>; Thu, 22 Aug 2002 12:34:53 +0100
Received: from listman.spamassassin.taint.org (localhost.localdomain [127.0.0.1]) by
    listman.redhat.com (Postfix) with ESMTP id 8386540858; Thu, 22 Aug 2002
    07:35:02 -0400 (EDT)
Delivered-To: exmh-workers@listman.spamassa

In [5]:
import sys
sys.getfilesystemencoding()

'utf-8'

In [6]:
stream = open(EXAMPLE_FILE, encoding='latin-1')

is_body = False
lines = []

for line in stream:
    if is_body:
        lines.append(line)
    elif line == '\n':
        is_body = True
        
stream.close()

email_body = '\n'.join(lines)
print(email_body)



Dear Mr Still



Good tidings to you and all your staff for the festive season ahead (Christmas).

Now to the crux of the matter-in-hand: I am a fully qualified Santa Claus and am wondering whether you might consider me to run my own "Santa's Grotto" in your store.

But WAIT! You're probably thinking: "What makes him so special?"

Well, first of all, I have made several changes to the characterisation of Father Christmas. Rather than greeting the children with shouts of "Ho, ho, ho!" I prefer to whisper the phrase "Dependence is not unfathomable in this cruel world we live in". In addition, my gifts are ALL hand-made, ranging from felt hoops to vanilla-pod holders.

You will note also, from the enclosed sketch, that I have radically redesigned Santa's outfit and have renamed my character "Lord Buckles". Would you be interested in employing me? I promise NEVER to let you down.

I look forward to hearing from you.



Best wishes

Robin Cooper

[Excerpt from the book: The Timewaster Let

# Generator Functions

In [7]:
def generate_squares(N):
    for my_number in range(N):
        yield my_number ** 2

In [8]:
for i in generate_squares(5):
    print(i, end=' -> ')

0 -> 1 -> 4 -> 9 -> 16 -> 

## E-mail Body Extraction

In [9]:
def email_body_generator(path):
    
    for root, pathname, filenames in walk(path):
        for file_name in filenames:
            
            filepath = join(root, file_name)
            
            stream = open(filepath, encoding='latin-1')

            is_body = False
            lines = []

            for line in stream:
                if is_body:
                    lines.append(line)
                elif line == '\n':
                    is_body = True

            stream.close()

            email_body = '\n'.join(lines)
            
            yield file_name, email_body

In [10]:
def df_from_directory(path, classification):
    rows = []
    row_names = []
    
    for file_name, email_body in email_body_generator(path):
        rows.append({'MESSAGE': email_body, 'CATEGORY': classification})
        row_names.append(file_name)
        
    return pd.DataFrame(rows, index=row_names)

In [11]:
spam_emails = df_from_directory(SPAM_1_PATH, SPAM_CAT)
spam_emails = spam_emails.append(df_from_directory(SPAM_2_PATH, SPAM_CAT))
spam_emails.head()

,MESSAGE,CATEGORY
00196.dd21040c7757d477c967ae71b537810e,"<html>\n\n\n\n<head>\n\n<meta http-equiv=3D""Co...",1
00056.c56d61cadd81b4ade0030c8dee384704,Note: \n\nThis is NOT SPAM!! This is NOT Unsol...,1
00219.eaf6c0ff67706c784f67f5c1225028a1,------=_NextPart_000_00D1_61B37D4A.A0476B56\n\...,1
00158.9c8bf53ed738031b4bfab819c4b3ef13,Wind up your mates today! Please visit http://...,1
00287.b0495a4dbdff36654c3b3ee2f92bdbf3,This is a multi-part message in MIME format.\n...,1


In [12]:
spam_emails.shape

(1898, 2)

In [13]:
ham_emails = df_from_directory(EASY_NONSPAM_1_PATH, HAM_CAT)
ham_emails = spam_emails.append(df_from_directory(EASY_NONSPAM_2_PATH, HAM_CAT))
ham_emails.head()

,MESSAGE,CATEGORY
00196.dd21040c7757d477c967ae71b537810e,"<html>\n\n\n\n<head>\n\n<meta http-equiv=3D""Co...",1
00056.c56d61cadd81b4ade0030c8dee384704,Note: \n\nThis is NOT SPAM!! This is NOT Unsol...,1
00219.eaf6c0ff67706c784f67f5c1225028a1,------=_NextPart_000_00D1_61B37D4A.A0476B56\n\...,1
00158.9c8bf53ed738031b4bfab819c4b3ef13,Wind up your mates today! Please visit http://...,1
00287.b0495a4dbdff36654c3b3ee2f92bdbf3,This is a multi-part message in MIME format.\n...,1


In [14]:
ham_emails.shape

(3298, 2)

In [15]:
data = pd.concat([spam_emails, ham_emails])
print('Shape of the entire dataset is:', data.shape)
data.head()

Shape of the entire dataset is: (5196, 2)


,MESSAGE,CATEGORY
00196.dd21040c7757d477c967ae71b537810e,"<html>\n\n\n\n<head>\n\n<meta http-equiv=3D""Co...",1
00056.c56d61cadd81b4ade0030c8dee384704,Note: \n\nThis is NOT SPAM!! This is NOT Unsol...,1
00219.eaf6c0ff67706c784f67f5c1225028a1,------=_NextPart_000_00D1_61B37D4A.A0476B56\n\...,1
00158.9c8bf53ed738031b4bfab819c4b3ef13,Wind up your mates today! Please visit http://...,1
00287.b0495a4dbdff36654c3b3ee2f92bdbf3,This is a multi-part message in MIME format.\n...,1


In [16]:
data.tail()

,MESSAGE,CATEGORY
00846.a603afcbab0796cbe45d3be854562598,"On Tue, 23 Jul 2002, Jim Whitehead wrote:\n\n\...",0
00424.e39b1db8cf5575572abb4482fd3fced3,wintermute wrote:\n\n>>Anyone know where in Ir...,0
00374.957837ba252b473057711e3c4dbd1e26,"hey,\n\n\n\nAFAIK it isn't hard at all to crea...",0
01101.ff91c2c8fb18ed6e300ed2ac699f8ae4,"Evidently, I have permission to pass this alon...",0
01266.94f7e1cce0ec1935ca75d232e4dc684c,"On Mon, 19 Aug 2002 16:44:30 +0200, Matthias S...",0


# Data Cleaning: Checking for Missing Values

In [17]:
# Check if any messages bodies ar null
data.MESSAGE.isnull().values.any()

False

In [18]:
type("")

str

In [19]:
len("")

0

In [20]:
my_var = None

In [21]:
type(my_var)

NoneType

In [22]:
# Check if there are empty e-mails (sting length zero)
(data.MESSAGE.str.len() == 0).any()

True

In [23]:
(data.MESSAGE.str.len() == 0).sum()

4

In [24]:
# Check the number of entries with Null/None values
data.MESSAGE.isnull().sum() + (data.MESSAGE.str.len() == 0).sum()

4

In [25]:
data[data.MESSAGE.str.len() == 0].index

Index(['cmds', 'cmds', 'cmds', 'cmds'], dtype='object')

# Remove System File Entries from Dataframe

In [26]:
data.drop(['cmds'], inplace=True)
data.shape

(5192, 2)

# Add Document IDs to track E-mails in Dataset

In [27]:
document_ids = range(len(data.index))
data['DOC_ID'] = document_ids

In [28]:
data.head()

,MESSAGE,CATEGORY,DOC_ID
00196.dd21040c7757d477c967ae71b537810e,"<html>\n\n\n\n<head>\n\n<meta http-equiv=3D""Co...",1,0
00056.c56d61cadd81b4ade0030c8dee384704,Note: \n\nThis is NOT SPAM!! This is NOT Unsol...,1,1
00219.eaf6c0ff67706c784f67f5c1225028a1,------=_NextPart_000_00D1_61B37D4A.A0476B56\n\...,1,2
00158.9c8bf53ed738031b4bfab819c4b3ef13,Wind up your mates today! Please visit http://...,1,3
00287.b0495a4dbdff36654c3b3ee2f92bdbf3,This is a multi-part message in MIME format.\n...,1,4


In [30]:
data['FILE_NAME'] = data.index
data.set_index('DOC_ID', inplace=True)
data.head()

,MESSAGE,CATEGORY,FILE_NAME
DOC_ID,,,
0,"<html>\n\n\n\n<head>\n\n<meta http-equiv=3D""Co...",1,00196.dd21040c7757d477c967ae71b537810e
1,Note: \n\nThis is NOT SPAM!! This is NOT Unsol...,1,00056.c56d61cadd81b4ade0030c8dee384704
2,------=_NextPart_000_00D1_61B37D4A.A0476B56\n\...,1,00219.eaf6c0ff67706c784f67f5c1225028a1
3,Wind up your mates today! Please visit http://...,1,00158.9c8bf53ed738031b4bfab819c4b3ef13
4,This is a multi-part message in MIME format.\n...,1,00287.b0495a4dbdff36654c3b3ee2f92bdbf3


In [31]:
data.tail()

,MESSAGE,CATEGORY,FILE_NAME
DOC_ID,,,
5187,"On Tue, 23 Jul 2002, Jim Whitehead wrote:\n\n\...",0,00846.a603afcbab0796cbe45d3be854562598
5188,wintermute wrote:\n\n>>Anyone know where in Ir...,0,00424.e39b1db8cf5575572abb4482fd3fced3
5189,"hey,\n\n\n\nAFAIK it isn't hard at all to crea...",0,00374.957837ba252b473057711e3c4dbd1e26
5190,"Evidently, I have permission to pass this alon...",0,01101.ff91c2c8fb18ed6e300ed2ac699f8ae4
5191,"On Mon, 19 Aug 2002 16:44:30 +0200, Matthias S...",0,01266.94f7e1cce0ec1935ca75d232e4dc684c
